# ONNX Inference.ipynb

## Setting Up Necessary Things

In [1]:
# Jupyter Notebook Magic Command - Auto Reloading
%reload_ext autoreload
%autoreload 2

# Jupyter Notebook Magic Command - Inline Plotting
%matplotlib inline

In [2]:
# Ignore All Warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
# NVIDIA Status
! nvidia-smi

Wed Aug 30 15:21:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/TC.Video.Game

/content/drive/MyDrive/TC.Video.Game


## Necessary Imports

In [6]:
! pip install -q onnxruntime onnx onnxruntime-gpu onnxruntime_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 99.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.2 MB/s eta 0:00:00


In [7]:
! pip install -q transformers sentencepiece ohmeow-blurr nbdev fastai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 113.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
import torch
from transformers import AutoModelForSequenceClassification, AutoConfig
from fastai.text.all import *
from blurr.text.data.all import *
from blurr.text.modeling.all import *

from tqdm.notebook import tqdm
import numpy as np

## Data Cleaning

In [9]:
# Load Data Into DataFrame
df = pd.read_csv("data/raw_data/game_details_original.csv")
df.head()

,Name,Summary,Genres
0,The Legend of Zelda: Ocarina of Time,"As a young boy, Link is tricked by Ganondorf, the King of the Gerudo Thieves. The evil human uses Link to gain access to the Sacred Realm, where he places his tainted hands on Triforce and transforms the beautiful Hyrulean landscape into a barren wasteland. Link is determined to fix the problems he helped to create, so with the help of Rauru he travels through time gathering the powers of the Seven Sages.","['Action Adventure', 'Fantasy']"
1,Tony Hawk's Pro Skater 2,"As most major publishers' development efforts shift to any number of next-generation platforms, Tony Hawk 2 will likely stand as one of the last truly fantastic games to be released on the PlayStation.","['Sports', 'Alternative', 'Skateboarding']"
2,Grand Theft Auto IV,"[Metacritic's 2008 PS3 Game of the Year; Also known as ""GTA IV""] What does the American Dream mean today? For Niko Belic, fresh off the boat from Europe. It's the hope he can escape his past. For his cousin, Roman, it is the vision that together they can find fortune in Liberty City, gateway to the land of opportunity. As they slip into debt and are dragged into a criminal underworld by a series of shysters, thieves and sociopaths, they discover that the reality is very different from the dream in a city that worships money and status, and is heaven for those who have them an a living nigh...","['Action Adventure', 'Modern', 'Modern', 'Open-World']"
3,SoulCalibur,"This is a tale of souls and swords, transcending the world and all its history, told for all eternity... The greatest weapons-based fighter returns, this time on Sega Dreamcast. Soul Calibur unleashes incredible graphics, fantastic fighters, and combos so amazing they'll make your head spin!","['Action', 'Fighting', '3D']"
4,Grand Theft Auto IV,"[Metacritic's 2008 Xbox 360 Game of the Year; Also known as ""GTA IV""] What does the American Dream mean today? For Niko Belic, fresh off the boat from Europe. It's the hope he can escape his past. For his cousin, Roman, it is the vision that together they can find fortune in Liberty City, gateway to the land of opportunity. As they slip into debt and are dragged into a criminal underworld by a series of shysters, thieves and sociopaths, they discover that the reality is very different from the dream in a city that worships money and status, and is heaven for those who have them an a living...","['Action Adventure', 'Modern', 'Modern', 'Open-World']"


In [10]:
# Shape of the DataFrame
df.shape

(20406, 3)

In [11]:
genres_list = df["Genres"].to_list()    # list of all the genres
genres_count = {}                       # count of all the genres

for genres in genres_list:
  try:
    genre_list = eval(genres)

    for genre in genre_list:
      if genre in genres_count.keys():
        genres_count[genre] += 1
      else:
        genres_count[genre] = 1
  except:
    continue

In [12]:
print("Total Number of Genres: ", len(genres_count))

sorted_genres_count = dict(sorted(genres_count.items(), key=lambda item: item[1], reverse=True))
print("Individual Genres Count: \n", sorted_genres_count)

Total Number of Genres:  178
Individual Genres Count: 
 {'Action': 7097, 'General': 7035, 'Shooter': 3549, 'Action Adventure': 3424, 'Platformer': 2371, 'Arcade': 2355, 'Role-Playing': 2341, '3D': 2127, 'Adventure': 1995, '2D': 1979, 'Miscellaneous': 1927, 'Strategy': 1905, 'First-Person': 1875, 'Sci-Fi': 1779, 'Fantasy': 1729, 'Sports': 1727, 'Modern': 1569, 'Puzzle': 1376, 'Action RPG': 1300, 'Racing': 1239, 'Sim': 1162, 'Simulation': 1160, 'Third-Person': 1040, 'Traditional': 1015, 'Driving': 971, 'Historic': 915, 'Fighting': 838, 'Automobile': 782, 'Real-Time': 772, 'Turn-Based': 740, 'Open-World': 736, "Beat-'Em-Up": 713, 'Team': 619, 'Compilation': 546, 'Survival': 544, 'Individual': 450, "Shoot-'Em-Up": 433, 'Point-and-Click': 430, 'Horror': 419, 'Japanese-Style': 415, 'Music': 411, 'Tactical': 405, 'Tactics': 392, 'Combat': 382, 'Linear': 353, 'Console-style RPG': 346, 'Rhythm': 340, 'Management': 334, 'GT / Street': 283, 'Western-Style': 279, 'Soccer': 276, 'Scrolling': 264, '

### Removing Rare Genres

In [13]:
threshold = int(len(df) * 0.05)

rare_genres = [item for item, count in genres_count.items() if count < threshold]

print("Total Rare Genres: ", len(rare_genres))
print("Rare Genres: ", rare_genres)

Total Rare Genres:  155
Rare Genres:  ['Alternative', 'Skateboarding', 'Open-World', 'Fighting', 'Western-Style', 'Traditional', 'Football', 'Linear', 'Horror', 'Compilation', 'Visual Novel', 'Driving', 'GT / Street', 'PC-style RPG', 'Historic', 'Japanese-Style', 'Stacking', 'Console-style RPG', 'Survival', 'Turn-Based', '4X', 'Real-Time', 'Roguelike', 'Management', 'Government', '', 'Metroidvania', 'Command', 'Snowboarding', 'Sandbox', 'Soccer', 'Kart', 'Rhythm', 'Music', "Beat-'Em-Up", 'Basketball', 'Application', 'Automobile', 'Ice Hockey', 'Virtual Life', 'Virtual', 'Wargame', 'Team', 'Tactical', 'Tennis', 'Tactics', 'Point-and-Click', 'Snow / Water', 'Massively Multiplayer', 'Flight', 'Civilian', 'Massively Multiplayer Online', 'Board / Card Game', 'Card Battle', 'Formula One', 'WWII', 'Combat', 'Space', 'Small Spaceship', 'Car Combat', 'Baseball', 'Individual', 'Golf', 'Skate / Skateboard', 'Motorcycle', 'Street', 'Business / Tycoon', 'Dancing', 'Wrestling', 'Text', 'Submarine', 

In [14]:
genres_list = df["Genres"].to_list()
revised_genre_list = []
indices_to_drop = []

for idx, genres in enumerate(genres_list):
  try:
    genre_list = eval(genres)
    revised_genres = []

    for genre in genre_list:
      if genre not in rare_genres:
        revised_genres.append(genre)

    if len(revised_genres) == 0:
      indices_to_drop.append(idx)
    else:
      revised_genres_set = set(revised_genres)
      revised_genre_list.append(list(revised_genres_set))
  except:
    indices_to_drop.append(idx)

In [15]:
df = df.drop(indices_to_drop).reset_index(drop=True)
df.shape

(19788, 3)

In [16]:
df["Revised Genres"] = revised_genre_list

In [17]:
revised_genres_list = df["Revised Genres"].to_list()
revised_genre_count = {}

for genres in revised_genres_list:
  genre_list = genres

  for genre in genre_list:
    if genre in revised_genre_count.keys():
      revised_genre_count[genre] += 1
    else:
      revised_genre_count[genre] = 1

In [18]:
print("Total Number of Revised Genres: ", len(revised_genre_count))

sorted_revised_genre_count = dict(sorted(revised_genre_count.items(), key=lambda item: item[1], reverse=True))
print("Individual Revised Genres Count: \n", sorted_revised_genre_count)

Total Number of Revised Genres:  23
Individual Revised Genres Count: 
 {'Action': 7037, 'General': 5579, 'Action Adventure': 3424, 'Shooter': 2513, 'Role-Playing': 2341, 'Arcade': 2186, 'Adventure': 1995, 'Miscellaneous': 1927, 'Strategy': 1905, 'Platformer': 1867, '3D': 1818, 'First-Person': 1811, 'Sports': 1727, '2D': 1590, 'Racing': 1239, 'Simulation': 1160, 'Sci-Fi': 1140, 'Fantasy': 1092, 'Action RPG': 1049, 'Third-Person': 1028, 'Modern': 998, 'Sim': 805, 'Puzzle': 751}


In [19]:
encode_genre_types = { key: idx for idx, (key, value) in enumerate(revised_genre_count.items())}

with open("encrypted/original/genre_types_encoded_original.json", "w") as fp:
  json.dump(encode_genre_types, fp)

In [20]:
encode_genre_types

{'Action Adventure': 0,
 'Fantasy': 1,
 'Sports': 2,
 'Modern': 3,
 '3D': 4,
 'Action': 5,
 'Platformer': 6,
 'Role-Playing': 7,
 'General': 8,
 'Sci-Fi': 9,
 'First-Person': 10,
 'Shooter': 11,
 'Sim': 12,
 'Arcade': 13,
 'Miscellaneous': 14,
 'Action RPG': 15,
 'Adventure': 16,
 'Racing': 17,
 '2D': 18,
 'Third-Person': 19,
 'Puzzle': 20,
 'Strategy': 21,
 'Simulation': 22}

In [21]:
# We need this because for multilabel classification all genres have possibility to be present in the predictions
categorical_genre_list = []
revised_genres_list = df["Revised Genres"].to_list()

for revised_genres in revised_genres_list:
  categorical_list = [0] * len(encode_genre_types)
  genre_list = revised_genres

  for genre in genre_list:
    if genre in encode_genre_types:
      genre_type_index = encode_genre_types[genre]
      categorical_list[genre_type_index] = 1

  categorical_genre_list.append(categorical_list)

categorical_genre_list[3][:23]

[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [22]:
df["Genre Cat List"] = categorical_genre_list

In [23]:
df.head()

,Name,Summary,Genres,Revised Genres,Genre Cat List
0,The Legend of Zelda: Ocarina of Time,"As a young boy, Link is tricked by Ganondorf, the King of the Gerudo Thieves. The evil human uses Link to gain access to the Sacred Realm, where he places his tainted hands on Triforce and transforms the beautiful Hyrulean landscape into a barren wasteland. Link is determined to fix the problems he helped to create, so with the help of Rauru he travels through time gathering the powers of the Seven Sages.","['Action Adventure', 'Fantasy']","[Action Adventure, Fantasy]","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,Tony Hawk's Pro Skater 2,"As most major publishers' development efforts shift to any number of next-generation platforms, Tony Hawk 2 will likely stand as one of the last truly fantastic games to be released on the PlayStation.","['Sports', 'Alternative', 'Skateboarding']",[Sports],"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,Grand Theft Auto IV,"[Metacritic's 2008 PS3 Game of the Year; Also known as ""GTA IV""] What does the American Dream mean today? For Niko Belic, fresh off the boat from Europe. It's the hope he can escape his past. For his cousin, Roman, it is the vision that together they can find fortune in Liberty City, gateway to the land of opportunity. As they slip into debt and are dragged into a criminal underworld by a series of shysters, thieves and sociopaths, they discover that the reality is very different from the dream in a city that worships money and status, and is heaven for those who have them an a living nigh...","['Action Adventure', 'Modern', 'Modern', 'Open-World']","[Modern, Action Adventure]","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,SoulCalibur,"This is a tale of souls and swords, transcending the world and all its history, told for all eternity... The greatest weapons-based fighter returns, this time on Sega Dreamcast. Soul Calibur unleashes incredible graphics, fantastic fighters, and combos so amazing they'll make your head spin!","['Action', 'Fighting', '3D']","[3D, Action]","[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,Grand Theft Auto IV,"[Metacritic's 2008 Xbox 360 Game of the Year; Also known as ""GTA IV""] What does the American Dream mean today? For Niko Belic, fresh off the boat from Europe. It's the hope he can escape his past. For his cousin, Roman, it is the vision that together they can find fortune in Liberty City, gateway to the land of opportunity. As they slip into debt and are dragged into a criminal underworld by a series of shysters, thieves and sociopaths, they discover that the reality is very different from the dream in a city that worships money and status, and is heaven for those who have them an a living...","['Action Adventure', 'Modern', 'Modern', 'Open-World']","[Modern, Action Adventure]","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [24]:
# Drop Duplicate Summary from the DataFrame
df = df.drop_duplicates(subset = "Summary", keep='first')

In [25]:
# Drop Genres Column
df.drop(["Genres"], axis = 1, inplace = True)

In [26]:
df = df.dropna().reset_index(drop=True)

In [27]:
df.isna().sum()

Name              0
Summary           0
Revised Genres    0
Genre Cat List    0
dtype: int64

In [28]:
# Description of The DataFrame
df.describe(include="object")

,Name,Summary,Revised Genres,Genre Cat List
count,15275,15275,15275,15275
unique,12584,15275,474,434
top,Destroy All Humans!,"As a young boy, Link is tricked by Ganondorf, the King of the Gerudo Thieves. The evil human uses Link to gain access to the Sacred Realm, where he places his tainted hands on Triforce and transforms the beautiful Hyrulean landscape into a barren wasteland. Link is determined to fix the problems he helped to create, so with the help of Rauru he travels through time gathering the powers of the Seven Sages.",[Action Adventure],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
freq,6,1,688,688


In [29]:
labels = list(encode_genre_types.keys())

print("Total Labels: ", len(labels))
print("Labels: ", labels)

Total Labels:  23
Labels:  ['Action Adventure', 'Fantasy', 'Sports', 'Modern', '3D', 'Action', 'Platformer', 'Role-Playing', 'General', 'Sci-Fi', 'First-Person', 'Shooter', 'Sim', 'Arcade', 'Miscellaneous', 'Action RPG', 'Adventure', 'Racing', '2D', 'Third-Person', 'Puzzle', 'Strategy', 'Simulation']


## Data Split

In [30]:
splitter = RandomSplitter(valid_pct=0.1, seed=42)

train_ids, valid_ids = splitter(df)

print("Train Len: ", len(train_ids))
print("Valid Len: ", len(valid_ids))

Train Len:  13748
Valid Len:  1527


In [31]:
valid_df = df.loc[valid_ids]

print("Valid Data: ")
valid_df.head()

Valid Data: 


,Name,Summary,Revised Genres,Genre Cat List
12017,Wii Music,"Wii Music includes many other modes besides the main band jams, including several musical games and an enhanced video playback mode for recorded jams. Play it again: Use the playback mode to see your jam recordings brought to life with dramatic camera angles. Pick up the baton: Command an orchestra in the conducting game where you’ll wave the Wii Remote controller like a conductor’s baton to lead a Mii orchestra through orchestrated music. Make them play quickly, slowly, strongly or gently. Ring a bell? Play a handbells game where you’ll swing your Wii Remote and Nunchuk controllers to pla...","[Action, 3D, Role-Playing, Platformer, Action RPG]","[0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
9726,Roller Champions,"Roller Champions is a 3v3 sports game where you go head-to-head against the opposing team through speed, wit, crushing tackles, and daring dodges! Be the first team to reach five points by throwing the ball through the hoop!","[3D, Action, Platformer]","[0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4509,NHL Hitz 20-02,"NHL® Hitz 20-02 is an all-new adrenaline-style, 4-on-4 hockey videogame. In true NHL® fashion, players skate full-speed into the boards to retrieve a loose puck, sacrifice their bodies by sliding in front of a 125MPH slapshot and throw down their gloves to defend their team. NHL® Hitz 20-02 features an interactive 3D crowd that reacts based on the action on the ice - crowds will bang on the glass, throw hats on ice and walk out on bad games.","[Strategy, Action, General]","[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]"
9753,Lone Ruin,"LONE RUIN is a spell-based roguelike twin-stick shooter with a focus on replayability. Play as an explorer who seeks a mysterious ancient power and venture in a ruined magical city, built atop a source of magic used by olden mages to power and transform themselves. Dive deeper and deeper, battling your way through twisted creatures, utilising your very own magic abilities to ultimately reach the center of the Lone Ruin.","[3D, Action, Platformer]","[0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3447,Dungeon Defenders,"Dungeon Defenders is an original hybrid of two hot genres: Tower Defense and online Action-RPG! The game puts players in the role of one of four distinctive hero classes, tasked with fending off hordes of invading beasties by strategically summoning a variety of Defenses & Traps throughout their castle. But as far more than a strategy game, players directly participate in the action-packed combat with their hero characters, while upgrading their statistics, gathering tons of loot and inventory items, customizing & upgrading equipment, and developing unique class abilities. All of this can ...","[Action Adventure, General]","[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


*We will be using `valid_df` for all inference testing*

## Fastai and Blurr Inference

In [32]:
model_path = "models/original/game-ml-classifier-stage-original-1.pkl"

learner_inf = load_learner(model_path)

In [33]:
learner_inf.blurr_predict("Action Adventure")

[{'labels': [],
  'scores': [],
  'class_indices': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'class_labels': ['Action Adventure', 'Fantasy', 'Sports', 'Modern', '3D', 'Action', 'Platformer', 'General', 'Role-Playing', 'Shooter', 'First-Person', 'Sci-Fi', 'Arcade', 'Miscellaneous', 'Adventure', '2D', 'Strategy'],
  'probs': [0.17264220118522644,
   0.053905803710222244,
   0.09085952490568161,
   0.050589557737112045,
   0.09221623837947845,
   0.36981654167175293,
   0.09486036747694016,
   0.2966024875640869,
   0.12710925936698914,
   0.13066481053829193,
   0.09650612622499466,
   0.059811439365148544,
   0.11134862154722214,
   0.10245078802108765,
   0.10579240322113037,
   0.08191661536693573,
   0.10444975644350052]}]

In [34]:
learner_inf.blurr_predict("random placeholder")[0]['labels']

[]

## Evaluation

In [35]:
from sklearn import metrics

def metric_measures(test_df, preds):

  targets = [np.asarray(target) for target in test_df['Genre Cat List'].to_list()]
  outputs = [np.asarray(pred) for pred in preds]


  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')

  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

  return

In [36]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=len(valid_df)):
  desc = row['Summary']
  labels = learner_inf.blurr_predict(desc)[0]['labels']
  pred_genres = [0] * len(encode_genre_types)
  for label in labels:
    pred_genres[encode_genre_types[label]] = 1
  preds.append(pred_genres)

preds[0][:20]

  0%|          | 0/1527 [00:00<?, ?it/s]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [37]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.0
F1 Score (Macro) = 0.0


## Convert to ONNX

In [38]:
model_path = "models/original/game-ml-classifier-stage-original-1.pkl"
learner_inf = load_learner(model_path)

In [39]:
classifier = learner_inf.model.hf_model.eval()

torch.onnx.export(
    classifier,
    torch.LongTensor([[0] * 512]),
    'models/original/game-classifier-original.onnx',
    input_names=['input_ids'],
    output_names=['output'],
    opset_version=13,
    dynamic_axes={
        'input_ids': {0: 'batch_size', 1: 'sequence_len'},
        'output': {0: 'batch_size'}
    }
)

In [40]:
from onnxruntime.quantization import quantize_dynamic, QuantType

onnx_model_path = 'models/original/game-classifier-original.onnx'
quantized_onnx_model_path = 'models/original/game-classifier-quantized-original.onnx'

quantize_dynamic(
    onnx_model_path,
    quantized_onnx_model_path,
    weight_type=QuantType.QUInt8,
)

Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.5/attention/self/MatMul]
Ignore MatMul due to non constant

## ONNX Inference

### Normal ONNX

In [41]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_genre_types.keys())

inf_session = rt.InferenceSession('models/original/game-classifier-original.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [42]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['Summary']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_genres = [0] * len(encode_genre_types)
  for label in labels:
    pred_genres[encode_genre_types[label]] = 1
  preds.append(pred_genres)

  0%|          | 0/1527 [00:00<?, ?it/s]

In [43]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.0
F1 Score (Macro) = 0.0


### Quantized ONNX

In [44]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_genre_types.keys())

inf_session = rt.InferenceSession('models/original/game-classifier-quantized-original.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [45]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['Summary']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_genres = [0] * len(encode_genre_types)
  for label in labels:
    pred_genres[encode_genre_types[label]] = 1
  preds.append(pred_genres)

  0%|          | 0/1527 [00:00<?, ?it/s]

In [46]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.0
F1 Score (Macro) = 0.0
